In [73]:
import pandas as pd
import numpy as np

In [74]:
pop1 = pd.read_csv('/Users/caleboneel/Desktop/estimating-impact-of-opioids-2020-purlple-team/20_intermediate_files/countypopulations_cleancopy.csv')

vitality = pd.read_csv('/Users/caleboneel/Desktop/estimating-impact-of-opioids-2020-purlple-team/10_code/od_deaths_state.csv')

In [75]:
mergepop = pop1.copy()
mergepop = mergepop.rename(columns = {'FIPS':'County Code', 'State Abbr':'State'}) 
mergepop.drop(['State Name','Unnamed: 0','_merge'], axis=1, inplace = True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2000.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2001.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2002.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2016.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2017.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2018.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2019.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['State']== 'ND'].index, inplace=True)
mergepop

,County,Year,Population,County Code,State
3,Autauga County,2003.0,46800,1001,AL
4,Autauga County,2004.0,48366,1001,AL
5,Autauga County,2005.0,49676,1001,AL
6,Autauga County,2006.0,51328,1001,AL
7,Autauga County,2007.0,52405,1001,AL
...,...,...,...,...,...
62331,Weston County,2011.0,7142,56045,WY
62332,Weston County,2012.0,7077,56045,WY
62333,Weston County,2013.0,7136,56045,WY
62334,Weston County,2014.0,7138,56045,WY


In [76]:
vitality

,County,State,County Code,Year,Deaths
0,Acadia Parish,LA,22001,2003.0,11.0
1,Acadia Parish,LA,22001,2005.0,23.0
2,Acadia Parish,LA,22001,2006.0,19.0
3,Acadia Parish,LA,22001,2007.0,19.0
4,Acadia Parish,LA,22001,2009.0,11.0
...,...,...,...,...,...
7920,Yuma County,AZ,4027,2011.0,20.0
7921,Yuma County,AZ,4027,2012.0,32.0
7922,Yuma County,AZ,4027,2013.0,22.0
7923,Yuma County,AZ,4027,2014.0,35.0


In [77]:
combo = vitality.merge(mergepop, how = 'right', on = ['County Code', 'Year'], indicator = True)

In [78]:
combo

,County_x,State_x,County Code,Year,Deaths,County_y,Population,State_y,_merge
0,NaN,NaN,1001,2003.0,NaN,Autauga County,46800,AL,right_only
1,NaN,NaN,1001,2004.0,NaN,Autauga County,48366,AL,right_only
2,NaN,NaN,1001,2005.0,NaN,Autauga County,49676,AL,right_only
3,NaN,NaN,1001,2006.0,NaN,Autauga County,51328,AL,right_only
4,NaN,NaN,1001,2007.0,NaN,Autauga County,52405,AL,right_only
...,...,...,...,...,...,...,...,...,...
39827,NaN,NaN,56045,2011.0,NaN,Weston County,7142,WY,right_only
39828,NaN,NaN,56045,2012.0,NaN,Weston County,7077,WY,right_only
39829,NaN,NaN,56045,2013.0,NaN,Weston County,7136,WY,right_only
39830,NaN,NaN,56045,2014.0,NaN,Weston County,7138,WY,right_only


In [79]:
death_coef = combo[combo['_merge'] == 'both']

In [80]:
death_coef = death_coef[['State_x', 'County Code', 'Year', 'Deaths', 'Population']]
death_coef['death rate'] = death_coef['Deaths']/death_coef['Population']
death_coef

,State_x,County Code,Year,Deaths,Population,death rate
13,AL,1003,2003.0,10.0,151509,0.000066
14,AL,1003,2004.0,18.0,156266,0.000115
15,AL,1003,2005.0,14.0,162183,0.000086
16,AL,1003,2006.0,11.0,168121,0.000065
17,AL,1003,2007.0,24.0,172404,0.000139
...,...,...,...,...,...,...
39698,WY,56025,2012.0,12.0,78565,0.000153
39699,WY,56025,2013.0,14.0,81103,0.000173
39700,WY,56025,2014.0,17.0,81377,0.000209
39701,WY,56025,2015.0,13.0,82137,0.000158


In [81]:
coef = death_coef.groupby(['State_x', 'Year'], as_index = False).median()

In [82]:
#coef['death rate'] = coef['Deaths']/coef['Population']
coef.rename(columns = {'State_x':'State'}, inplace = True) 
coef

,State,Year,County Code,Deaths,Population,death rate
0,AL,2003.0,1073.0,26.0,398082.0,0.000066
1,AL,2004.0,1093.0,15.0,259328.5,0.000082
2,AL,2005.0,1097.0,13.0,175731.0,0.000086
3,AL,2006.0,1099.0,15.0,180112.0,0.000099
4,AL,2007.0,1093.0,17.5,144400.5,0.000134
...,...,...,...,...,...,...
610,WY,2010.0,56023.0,10.5,83853.0,0.000126
611,WY,2012.0,56023.0,13.5,86631.0,0.000156
612,WY,2013.0,56023.0,14.5,88405.0,0.000165
613,WY,2014.0,56025.0,13.0,81377.0,0.000209


In [83]:
fl = coef[coef['State']=='FL']
fl

,State,Year,County Code,Deaths,Population,death rate
104,FL,2003.0,12081.0,29.0,285321.0,0.000110
105,FL,2004.0,12085.5,25.0,293592.5,0.000119
106,FL,2005.0,12082.0,30.0,300948.5,0.000122
107,FL,2006.0,12085.0,45.0,312621.0,0.000162
108,FL,2007.0,12084.0,33.5,293651.0,0.000152
109,FL,2008.0,12083.0,38.0,296854.0,0.000152
110,FL,2009.0,12083.0,53.0,318485.0,0.000160
111,FL,2010.0,12084.0,39.5,297873.5,0.000152
112,FL,2011.0,12084.0,36.5,289908.5,0.000158
113,FL,2012.0,12081.0,29.0,303423.0,0.000121


# Markdown incase reuse
wyoming = coef[coef['State_x'] == 'WY']
wyoming

wy = pop1.groupby(['State Abbr', 'Year'], as_index = False).sum()
wy1 = wy[wy['State Abbr'] == 'WY']
wy1

In [84]:
deathrate = coef.drop(['Deaths', 'Population', 'County Code'], axis=1)
deathrate

,State,Year,death rate
0,AL,2003.0,0.000066
1,AL,2004.0,0.000082
2,AL,2005.0,0.000086
3,AL,2006.0,0.000099
4,AL,2007.0,0.000134
...,...,...,...
610,WY,2010.0,0.000126
611,WY,2012.0,0.000156
612,WY,2013.0,0.000165
613,WY,2014.0,0.000209


In [85]:
combo['Merge?'] = combo['_merge']
combo.drop(['_merge'], axis=1, inplace = True)

In [86]:
combo.rename(columns = {'State_y':'State', 'County_y':'County'}, inplace = True)
combo.drop(['County_x', 'State_x'], axis = 1, inplace = True)
combo

,County Code,Year,Deaths,County,Population,State,Merge?
0,1001,2003.0,NaN,Autauga County,46800,AL,right_only
1,1001,2004.0,NaN,Autauga County,48366,AL,right_only
2,1001,2005.0,NaN,Autauga County,49676,AL,right_only
3,1001,2006.0,NaN,Autauga County,51328,AL,right_only
4,1001,2007.0,NaN,Autauga County,52405,AL,right_only
...,...,...,...,...,...,...,...
39827,56045,2011.0,NaN,Weston County,7142,WY,right_only
39828,56045,2012.0,NaN,Weston County,7077,WY,right_only
39829,56045,2013.0,NaN,Weston County,7136,WY,right_only
39830,56045,2014.0,NaN,Weston County,7138,WY,right_only


In [89]:
death = combo.merge(deathrate, how='left', on = ['State', 'Year',], indicator = True)

In [90]:
death['death rate'].isna().sum()
deathnan = death[death['_merge'] == 'left_only']

multiple = death.copy()
if ((multiple['Merge?'] == 'right_only') and (multiple['_merge'] == 'both')):
    deathcoef = round(multiple['Population'] * multiple['death rate'])
    multiple['Deaths'] = min(9, deathcoef)

multiple = death.copy()
multiple['deathcoef'] = round(multiple['Population'] * multiple['death rate'])
multiple.loc[multiple['Merge?'] == 'right_only', 'projdeaths'] = min(9, deathcoef)

In [91]:
deathnanyr = deathnan.groupby(['State','Year'], as_index = False).sum()
deathnanyr

,State,Year,County Code,Deaths,Population,death rate
0,MT,2004.0,1683136,0.0,930009,0.0
1,NE,2003.0,2891649,0.0,1738643,0.0
2,SD,2003.0,3040496,0.0,763729,0.0
3,SD,2004.0,3040496,0.0,770396,0.0
4,SD,2005.0,3040496,0.0,775493,0.0
5,SD,2006.0,3040496,0.0,783033,0.0
6,SD,2007.0,3040496,0.0,791623,0.0
7,SD,2008.0,3040496,0.0,799124,0.0
8,SD,2011.0,3040496,0.0,823579,0.0
9,SD,2012.0,3040496,0.0,833566,0.0


In [92]:
deathnanyr['State'].value_counts()


SD    8
VT    6
WY    6
NE    1
MT    1
Name: State, dtype: int64

In [93]:
multiple = death.copy()
missing =  multiple[multiple['State'].isin(['SD', 'VT', 'WY', 'NE', 'MT'])]  

In [94]:
SD = missing[missing['State']== 'SD']
VT = missing[missing['State']== 'VT']
WY = missing[missing['State']== 'WY']
NE = missing[missing['State']== 'NE']
MT = missing[missing['State']== 'MT']

In [95]:
SD = SD.groupby(['Year'], as_index = False).mean()
SDrate = SD['death rate'].mean()

VT = VT.groupby(['Year'], as_index = False).mean()
VTrate = VT['death rate'].mean()

WY = WY.groupby(['Year'], as_index = False).mean()
WYrate = WY['death rate'].mean()

NE = NE.groupby(['Year'], as_index = False).mean()
NErate = NE['death rate'].mean()

MT = MT.groupby(['Year'], as_index = False).mean()
MTrate = MT['death rate'].mean()

In [96]:
multcopy = multiple.copy()
multcopy

,County Code,Year,Deaths,County,Population,State,Merge?,death rate,_merge
0,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000066,both
1,1001,2004.0,NaN,Autauga County,48366,AL,right_only,0.000082,both
2,1001,2005.0,NaN,Autauga County,49676,AL,right_only,0.000086,both
3,1001,2006.0,NaN,Autauga County,51328,AL,right_only,0.000099,both
4,1001,2007.0,NaN,Autauga County,52405,AL,right_only,0.000134,both
...,...,...,...,...,...,...,...,...,...
39827,56045,2011.0,NaN,Weston County,7142,WY,right_only,NaN,left_only
39828,56045,2012.0,NaN,Weston County,7077,WY,right_only,0.000156,both
39829,56045,2013.0,NaN,Weston County,7136,WY,right_only,0.000165,both
39830,56045,2014.0,NaN,Weston County,7138,WY,right_only,0.000209,both


In [97]:
multcopy['death rate'] = np.where((multcopy.State == 'SD')&(multcopy['death rate'].isnull()),
                                 SDrate,
                                 multcopy['death rate'])

In [98]:
multcopy['death rate'] = np.where((multcopy.State == 'VT')&(multcopy['death rate'].isnull()),
                                 VTrate,
                                 multcopy['death rate'])

In [99]:
multcopy['death rate'] = np.where((multcopy.State == 'WY')&(multcopy['death rate'].isnull()),
                                 WYrate,
                                 multcopy['death rate'])

In [100]:
multcopy['death rate'] = np.where((multcopy.State == 'NE')&(multcopy['death rate'].isnull()),
                                 NErate,
                                 multcopy['death rate'])

In [101]:
multcopy['death rate'] = np.where((multcopy.State == 'MT')&(multcopy['death rate'].isnull()),
                                 MTrate,
                                 multcopy['death rate'])

In [102]:
SDnew = multcopy[multcopy['State'] == 'SD']

In [103]:
SDnew.groupby(['Year']).mean()

,County Code,Deaths,Population,death rate
Year,,,,
2003.0,46068.121212,NaN,11571.651515,0.000091
2004.0,46068.121212,NaN,11672.666667,0.000091
2005.0,46068.121212,NaN,11749.893939,0.000091
2006.0,46068.121212,NaN,11864.136364,0.000091
2007.0,46068.121212,NaN,11994.287879,0.000091
2008.0,46068.121212,NaN,12107.939394,0.000091
2009.0,46068.121212,14.0,12228.287879,0.000140
2010.0,46068.121212,11.0,12366.151515,0.000065
2011.0,46068.121212,NaN,12478.469697,0.000091


In [104]:
VTnew = multcopy[multcopy['State'] == 'VT']
VTnew

,County Code,Year,Deaths,County,Population,State,Merge?,death rate,_merge
35451,50001,2003.0,NaN,Addison County,36432,VT,right_only,0.000131,left_only
35452,50001,2004.0,NaN,Addison County,36580,VT,right_only,0.000131,left_only
35453,50001,2005.0,NaN,Addison County,36611,VT,right_only,0.000131,left_only
35454,50001,2006.0,NaN,Addison County,36758,VT,right_only,0.000098,both
35455,50001,2007.0,NaN,Addison County,36886,VT,right_only,0.000192,both
...,...,...,...,...,...,...,...,...,...
35628,50027,2011.0,NaN,Windsor County,56612,VT,right_only,0.000139,both
35629,50027,2012.0,NaN,Windsor County,56138,VT,right_only,0.000088,both
35630,50027,2013.0,NaN,Windsor County,55993,VT,right_only,0.000127,both
35631,50027,2014.0,NaN,Windsor County,55727,VT,right_only,0.000081,both


In [105]:
multcopy['death rate'].isnull().values.any()
multcopy

,County Code,Year,Deaths,County,Population,State,Merge?,death rate,_merge
0,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000066,both
1,1001,2004.0,NaN,Autauga County,48366,AL,right_only,0.000082,both
2,1001,2005.0,NaN,Autauga County,49676,AL,right_only,0.000086,both
3,1001,2006.0,NaN,Autauga County,51328,AL,right_only,0.000099,both
4,1001,2007.0,NaN,Autauga County,52405,AL,right_only,0.000134,both
...,...,...,...,...,...,...,...,...,...
39827,56045,2011.0,NaN,Weston County,7142,WY,right_only,0.000157,left_only
39828,56045,2012.0,NaN,Weston County,7077,WY,right_only,0.000156,both
39829,56045,2013.0,NaN,Weston County,7136,WY,right_only,0.000165,both
39830,56045,2014.0,NaN,Weston County,7138,WY,right_only,0.000209,both


In [106]:
#multcopy.drop(['deathcoef'], axis=1, inplace = True)

In [107]:
multcopy['deathcoef'] = round(multcopy['Population'] * multcopy['death rate'])


In [108]:
multcopy['projdeaths'] = np.where((multcopy.deathcoef < 9),
                                 multcopy.deathcoef,
                                 9)

In [109]:
multcopy

,County Code,Year,Deaths,County,Population,State,Merge?,death rate,_merge,deathcoef,projdeaths
0,1001,2003.0,NaN,Autauga County,46800,AL,right_only,0.000066,both,3.0,3.0
1,1001,2004.0,NaN,Autauga County,48366,AL,right_only,0.000082,both,4.0,4.0
2,1001,2005.0,NaN,Autauga County,49676,AL,right_only,0.000086,both,4.0,4.0
3,1001,2006.0,NaN,Autauga County,51328,AL,right_only,0.000099,both,5.0,5.0
4,1001,2007.0,NaN,Autauga County,52405,AL,right_only,0.000134,both,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
39827,56045,2011.0,NaN,Weston County,7142,WY,right_only,0.000157,left_only,1.0,1.0
39828,56045,2012.0,NaN,Weston County,7077,WY,right_only,0.000156,both,1.0,1.0
39829,56045,2013.0,NaN,Weston County,7136,WY,right_only,0.000165,both,1.0,1.0
39830,56045,2014.0,NaN,Weston County,7138,WY,right_only,0.000209,both,1.0,1.0


In [110]:
multcopy['CalculatedDeaths'] = np.where((multcopy.Deaths.isnull()),
                                 multcopy.projdeaths,
                                 multcopy.Deaths)

In [111]:
assert len(multcopy) == len(mergepop)
assert multcopy.projdeaths.isnull().values.any() == False

In [112]:
multcopy.drop(['Merge?', '_merge'], axis=1, inplace = True)

In [113]:
vitalitydf = multcopy.copy()

In [114]:
vitalitydf.to_csv('vitalitydf_allcounties', index=False)